In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
from os import listdir, makedirs
from os.path import join, isfile, isdir
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
from tqdm import tqdm

In [ ]:
root_folder = '/gdrive/My Drive/OA_Gihan/OA_all/OA activity data_La Trobe'
sleep_df_file = '/gdrive/My Drive/OA_Gihan/OA_all/OA activity data_La Trobe/Analytics_Scripts/Rashmika_Analysis/metadata/Copy of sleep_time_modified.xlsx'
# sleep_df_file = '/gdrive/My Drive/OA_Gihan/OA_all/OA activity data_La Trobe/Analytics_Scripts/Rashmika_Analysis/metadata/Copy of record_availability_final.csv'
outout_filename = '/gdrive/My Drive/OA activity data_La Trobe/Analytics_Scripts/Gihan_analysis/Data/metadata/record_availability_final_pal.csv'

In [ ]:
hkoa_df = pd.read_excel(sleep_df_file, sheet_name='HOA-KOA')
oa_df = pd.read_excel(sleep_df_file, sheet_name='OA')

In [ ]:
hkoa_df.head()

,ID,Participant,activpal_Start_time,activpal_End_time,activpal_Time_difference,actigraph_Start_time,actigraph_End_time,actigraph_Time_difference,Overlap,age_yr,sex,BMI,bodymass_kg,AG_data,AP_data,"Side_TKA (1=L,2=R)",slnumnew,nonwearmins,sleepmins,nonwearpct,justnonwear,justsleep,av_inbedtime,av_outofbed_time
0,0,HOA1,2019-05-06 14:59:46,2019-05-13 08:48:55,161h 49m 9d,2019-05-06 15:00:00,2019-05-13 07:59:59,160h 59m 59d,160h 59m 59d,52.0,female,76.86261,88.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22:00:00,06:00:00
1,1,HOA10,2019-06-24 13:59:47,2019-07-01 10:06:44,164h 6m 57d,2019-06-24 14:00:00,2019-07-01 07:59:59,161h 59m 59d,161h 59m 59d,51.0,female,42.41690,98.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21:00:00,09:00:00
2,2,HOA11,2019-07-01 13:59:47,2019-07-08 11:09:17,165h 9m 30d,2019-07-01 14:00:00,2019-07-08 07:59:59,161h 59m 59d,161h 59m 59d,56.0,male,30.09143,78.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20:00:00,06:00:00
3,3,HOA12,2019-07-01 15:59:47,2019-07-08 10:44:11,162h 44m 24d,2019-07-01 16:00:00,2019-07-08 07:59:59,159h 59m 59d,159h 59m 59d,61.0,female,27.43484,72.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20:00:00,05:00:00
4,5,HOA14,2019-07-15 14:59:49,2019-07-22 10:17:40,163h 17m 51d,2019-07-15 15:00:00,2019-07-22 07:59:59,160h 59m 59d,160h 59m 59d,55.0,female,40.53871,90.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20:00:00,05:00:00


### Process HKOA datasets

In [ ]:
hkoa_out_df = None

for user_id in tqdm(list(hkoa_df['Participant'].unique())):

  user_df = hkoa_df.loc[hkoa_df['Participant']==user_id]

  in_time = user_df.iloc[0]['av_inbedtime']
  i_h, i_m, i_s = str(in_time).split(':')
  i_h, i_m, i_s = int(i_h), int(i_m), int(i_s)
  out_time = user_df.iloc[0]['av_outofbed_time']
  o_h, o_m, o_s = str(out_time).split(':')
  o_h, o_m, o_s = int(o_h), int(o_m), int(o_s)

  d1 = user_df.iloc[0]['activpal_Start_time']
  d2 = user_df.iloc[0]['activpal_End_time']
  dd = [d1 + timedelta(days=x) for x in range((d2-d1).days + 2)]

  begin_times = []
  end_times = []
  for i, d in enumerate(dd):

    if i == 0:
      begin_time = d1
      end_time = d.replace(hour=i_h, minute=i_m, second=i_s)
    elif i == len(dd)-1:
      begin_time = d.replace(hour=o_h, minute=o_m, second=o_s)
      end_time = d2
    else:
      begin_time = d.replace(hour=o_h, minute=o_m, second=o_s)
      end_time = d.replace(hour=i_h, minute=i_m, second=i_s)

    if begin_time > end_time:
      print("Error in {} \n {} to {}".format(user_id, begin_time, end_time))
      continue

    begin_times.append(begin_time)
    end_times.append(end_time)

  newdf = pd.DataFrame(np.repeat(user_df.values, len(begin_times), axis=0))
  newdf.columns = user_df.columns
  newdf['begin_time_pal'] = begin_times
  newdf['end_time_pal'] = end_times

  del newdf['av_inbedtime']
  del newdf['av_outofbed_time']
  
  if hkoa_out_df is None:
    hkoa_out_df = newdf
  else:
    hkoa_out_df = hkoa_out_df.append(newdf, ignore_index=True)

 23%|██▎       | 22/96 [00:00<00:00, 104.93it/s]

Error in HOA2 
 2019-05-14 04:30:00 to 2019-05-13 15:00:03
Error in HOA24 
 2019-09-03 07:00:00 to 2019-09-02 16:59:58
Error in HOA26 
 2019-09-10 07:00:00 to 2019-09-09 09:00:02
Error in HOA27 
 2019-09-10 06:30:00 to 2019-09-09 08:59:57
Error in HOA30 
 2019-09-10 05:00:00 to 2019-09-09 16:00:10
Error in HOA35 
 2019-10-01 06:00:00 to 2019-09-30 15:00:09


 69%|██████▉   | 66/96 [00:00<00:00, 104.25it/s]

Error in KOA26 
 2019-07-28 05:00:00 to 2019-07-28 01:56:17
Error in KOA38 
 2019-09-03 08:00:00 to 2019-09-02 15:00:09
Error in KOA41 
 2019-09-03 07:30:00 to 2019-09-02 17:00:04
Error in KOA42 
 2019-09-03 05:00:00 to 2019-09-02 17:00:10
Error in KOA44 
 2019-09-10 08:00:00 to 2019-09-09 09:00:01
Error in KOA46 
 2019-09-10 08:00:00 to 2019-09-09 09:00:08
Error in KOA49 
 2019-09-10 07:00:00 to 2019-09-09 14:00:05
Error in KOA51 
 2019-09-17 07:30:00 to 2019-09-16 15:00:02


100%|██████████| 96/96 [00:00<00:00, 103.98it/s]

Error in KOA60 
 2019-10-15 04:30:00 to 2019-10-14 16:00:11
Error in KOA61 
 2019-10-15 09:00:00 to 2019-10-14 16:00:03
Error in KOA71 
 2019-10-21 22:59:47 to 2019-10-21 21:00:00


In [ ]:
hkoa_out_df.head()

,ID,Participant,activpal_Start_time,activpal_End_time,activpal_Time_difference,actigraph_Start_time,actigraph_End_time,actigraph_Time_difference,Overlap,age_yr,sex,BMI,bodymass_kg,AG_data,AP_data,"Side_TKA (1=L,2=R)",slnumnew,nonwearmins,sleepmins,nonwearpct,justnonwear,justsleep,begin_time_pal,end_time_pal
0,0,HOA1,2019-05-06 14:59:46,2019-05-13 08:48:55,161h 49m 9d,2019-05-06 15:00:00,2019-05-13 07:59:59,160h 59m 59d,160h 59m 59d,52,female,76.8626,88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-05-06 14:59:46,2019-05-06 22:00:00
1,0,HOA1,2019-05-06 14:59:46,2019-05-13 08:48:55,161h 49m 9d,2019-05-06 15:00:00,2019-05-13 07:59:59,160h 59m 59d,160h 59m 59d,52,female,76.8626,88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-05-07 06:00:00,2019-05-07 22:00:00
2,0,HOA1,2019-05-06 14:59:46,2019-05-13 08:48:55,161h 49m 9d,2019-05-06 15:00:00,2019-05-13 07:59:59,160h 59m 59d,160h 59m 59d,52,female,76.8626,88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-05-08 06:00:00,2019-05-08 22:00:00
3,0,HOA1,2019-05-06 14:59:46,2019-05-13 08:48:55,161h 49m 9d,2019-05-06 15:00:00,2019-05-13 07:59:59,160h 59m 59d,160h 59m 59d,52,female,76.8626,88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-05-09 06:00:00,2019-05-09 22:00:00
4,0,HOA1,2019-05-06 14:59:46,2019-05-13 08:48:55,161h 49m 9d,2019-05-06 15:00:00,2019-05-13 07:59:59,160h 59m 59d,160h 59m 59d,52,female,76.8626,88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-05-10 06:00:00,2019-05-10 22:00:00


In [ ]:
hkoa_out_df.to_csv('hkoa_pal.csv', index=None)

### Process OA data

In [ ]:
oa_out_df = None

for user_id in tqdm(list(oa_df['Participant'].unique())):

  user_df = oa_df.loc[oa_df['Participant']==user_id]

  all_start_time = user_df.iloc[0]['activpal_Start_time']
  all_end_time = user_df.iloc[0]['activpal_End_time']
  sleep_start_times = user_df['av_inbedtime'].tolist()
  sleep_end_times = user_df['av_outofbed_time'].tolist()

  sleep_end_times.insert(0, str(all_start_time))
  sleep_start_times.append(str(all_end_time))

  start_processed = []
  end_processed = []
  for b, e in zip(sleep_end_times, sleep_start_times):

    b = datetime.strptime(b, '%Y-%m-%d %H:%M:%S')
    e = datetime.strptime(e, '%Y-%m-%d %H:%M:%S')

    if (e-b).seconds//3600 < 1:
      continue

    start_processed.append(b)
    end_processed.append(e)

  # Create a new dataframe
  newdf = pd.DataFrame(np.repeat(user_df[0:1].values, len(start_processed), axis=0))
  newdf.columns = user_df.columns
  newdf['begin_time_pal'] = start_processed
  newdf['end_time_pal'] = end_processed

  del newdf['av_inbedtime']
  del newdf['av_outofbed_time']

  if oa_out_df is None:
    oa_out_df = newdf
  else:
    oa_out_df = oa_out_df.append(newdf, ignore_index=True)

100%|██████████| 82/82 [00:00<00:00, 104.69it/s]


In [ ]:
oa_out_df.head()

,ID,Participant,activpal_Start_time,activpal_End_time,activpal_Time_difference,actigraph_Start_time,actigraph_End_time,actigraph_Time_difference,Overlap,age_yr,sex,BMI,bodymass_kg,AG_data,AP_data,"Side_TKA (1=L,2=R)",slnumnew,nonwearmins,sleepmins,nonwearpct,justnonwear,justsleep,begin_time_pal,end_time_pal
0,106,OA1,2015-07-21 09:44:49,2015-07-27 10:27:59,144h 43m 10d,2015-07-20 09:48:00,2015-07-27 09:59:59,168h 11m 59d,144h 15m 10d,68,male,30.2,98,NaN,NaN,1,1,0,157,0,NaN,1,2015-07-21 09:44:49,2015-07-20 23:58:00
1,106,OA1,2015-07-21 09:44:49,2015-07-27 10:27:59,144h 43m 10d,2015-07-20 09:48:00,2015-07-27 09:59:59,168h 11m 59d,144h 15m 10d,68,male,30.2,98,NaN,NaN,1,1,0,157,0,NaN,1,2015-07-21 02:34:00,2015-07-22 00:26:00
2,106,OA1,2015-07-21 09:44:49,2015-07-27 10:27:59,144h 43m 10d,2015-07-20 09:48:00,2015-07-27 09:59:59,168h 11m 59d,144h 15m 10d,68,male,30.2,98,NaN,NaN,1,1,0,157,0,NaN,1,2015-07-22 03:15:00,2015-07-22 23:02:00
3,106,OA1,2015-07-21 09:44:49,2015-07-27 10:27:59,144h 43m 10d,2015-07-20 09:48:00,2015-07-27 09:59:59,168h 11m 59d,144h 15m 10d,68,male,30.2,98,NaN,NaN,1,1,0,157,0,NaN,1,2015-07-23 02:52:00,2015-07-24 23:26:00
4,106,OA1,2015-07-21 09:44:49,2015-07-27 10:27:59,144h 43m 10d,2015-07-20 09:48:00,2015-07-27 09:59:59,168h 11m 59d,144h 15m 10d,68,male,30.2,98,NaN,NaN,1,1,0,157,0,NaN,1,2015-07-25 03:44:00,2015-07-26 01:16:00


In [ ]:
oa_out_df.to_csv('oa_pal.csv', index=None)

### Combine both types of usrs

In [ ]:
final_df = hkoa_out_df.append(oa_out_df, ignore_index=True)
final_df.to_csv(outout_filename, index=None)